In [1]:
# would recommend using conda for the packages
import sys
import os
import io
from pgn_parser import parser, pgn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, History
import chess
import chess.pgn
import tqdm
import pandas as pd
import numpy as np
import random
from stockfish import Stockfish


In [2]:
# converts a FEN notation board into an 8-by-8 array containing all the pieces in their respective positions.
def board_fen_to_8x8(bf):
    our_board = list()
    long_board = list()
    long_board2 = list()
    # removes '/' splitters from FEN notation 
    for s in bf.split('/'):
        long_board+=s
    for l in long_board: 
        # replace all black spaces with 0's in board row
        if l.isnumeric():     
            i = int(l)
            while i>0:
                long_board2.append(0)
                i-=1
        else:
            long_board2.append(l)
    h = 0
    # create end 8-by-8 array with all pieces and 0's for blanks spaces
    for i in range(0,8):
        our_board.append([])
        for j in range(0,8):
            our_board[i].append([])
            our_board[i][j] = long_board2[h]
            h+=1
            
    return our_board


In [3]:
# replace piece letters in board with number between -6 and 6 (not 0)
def letter_to_number(letter):
    letter = str(letter)
    color = 1
    if letter.isupper(): color = -1
    return  color*((1*(letter.lower() == 'p'))+(2*(letter.lower() == 'n'))\
            +(3*(letter.lower() == 'b'))+(4*(letter.lower() == 'r'))\
            +(5*(letter.lower() == 'q'))+(6*(letter.lower() == 'k')))

In [4]:
# for every piece square, call letter_to_number function to replace it with an integer
def board_to_nums(board):
    b2 = board.copy()
    for i in range(0,8):
        for j in range(0,8):
            b2[i][j] = letter_to_number(board[i][j])
    return b2

In [5]:
def board_to_12_array(board):
    ta = [[],[],[],[],[],[],[],[],[],[],[],[]]
    # if its a 0, add 0 to all 12 arrays
    # if a 1,2,3,4,5,6 or negatives, add a 1 to the corresponding array and 0 to all others
    
    b = board.copy()
    # iterates through all rows in the boards
    for i in range(0,8):
        # add a new row to every board
        for m in range(0,12):
            ta[m].append([])
        # for each column, check if space is blank or filled by a white or black piece
        for j in range(0,8):
            # blank space
            if b[i][j] == 0:
                for k in range(0,12):
                    ta[k][i].append(0)
            # piece is white
            elif b[i][j] < 0:
                ta[-1*(b[i][j])+5][i].append(1)
                for k in range(0,12):
                    if k != (-1*(b[i][j])+5):
                        ta[k][i].append(0)
            # piece is black   
            else:
                ta[b[i][j]-1][i].append(1)
                for k in range(0,12):
                    if k != (b[i][j]-1):
                        ta[k][i].append(0)
        
            
    return ta

In [7]:
boards = []
num_games = 1000
gamepgn=open("lichess_12_2022.pgn",'r')

# go through first 1000 games from dataset
for i in range(0,num_games):
    game = chess.pgn.read_game(gamepgn)
    board = game.board()
    # after reading the next game, take each position's FEN and convert to board array
    # then add that board array to the dataset
    x = 1
    for move in game.mainline_moves():
        if x > 0:
            b = {}
            b['board'] = board_to_nums(board_fen_to_8x8(board.board_fen()))
            b['oneHboards'] = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
            boards.append(b)
        board.push(move)
        x = x * (-1)
boards = pd.DataFrame(boards)

print(len(boards))

test_boards = []

# generate test dataset for later
for i in range(0,100):
    game = chess.pgn.read_game(gamepgn)
    board = game.board()
    x = 1
    for move in game.mainline_moves():
        if x > 0:
            test_boards.append(board.board_fen())
        board.push(move)
        x = x * (-1)


43418


In [8]:
rows = []
with open("lichess_12_2022.pgn") as pgn:
    # iterate through all games and add the expected move outputs to the dataset for each position
    for game in tqdm.tqdm(range(num_games)):
        game = chess.pgn.read_game(pgn)
        x = 1
        for move in game.mainline_moves():
            if x > 0:
                # convert to the format that we want (4 numbers between 0 and 1 corresponding to row/col of start/end square)
                u=list(str(move.uci()).strip(""))
                squares=[]
                for i in range(0,4):
                    if u[i].isnumeric():
                        squares.append(float(u[i])/8)
                    else:
                        squares.append(float(ord(u[i])-96)/8) 
                row={}
                row['moves'] = squares
                rows.append(row)
            x = x * (-1)
                
moves = pd.DataFrame(rows)

# finish the master dataset
master = pd.concat([boards, moves], axis=1, join="inner")
print(master)

master.to_csv("loadedgames.csv", index = False)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 787.40it/s]


                                                   board   
0      [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 1, 1, 1, ...  \
1      [[4, 2, 3, 5, 6, 3, 2, 4], [1, 1, 1, 0, 1, 1, ...   
2      [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 1, 0, 1, 1, ...   
3      [[4, 2, 3, 5, 6, 3, 0, 4], [1, 1, 0, 0, 1, 1, ...   
4      [[4, 2, 0, 5, 6, 3, 0, 4], [1, 1, 0, 0, 1, 1, ...   
...                                                  ...   
43413  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, ...   
43414  [[0, 0, 0, 0, 0, 0, 0, 0], [-6, 0, 0, 0, 0, 0,...   
43415  [[0, -6, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0,...   
43416  [[0, 0, 0, 0, 0, 0, 0, 0], [0, -6, 0, 0, 0, 0,...   
43417  [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, ...   

                                              oneHboards   
0      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1,...  \
1      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
2      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1,...   
3      [[[0, 0, 0, 0, 0, 0, 0, 0], [1, 

In [9]:
# model attempt 1
inp = keras.layers.Input(shape=(12,8,8))

# formatting model with 2 Conv2D layers with MaxPooling, and one Dense layer before the output
m = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv1")(inp)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="valid", name="mPool1")(m)
m = keras.layers.Conv2D(filters=64, kernel_size=3, strides=(1,1), padding="same", activation='relu', name="conv2")(m)
m = keras.layers.MaxPooling2D(pool_size=(2,2), padding="valid", name="mPool2")(m)
m = keras.layers.Flatten()(m)
m = keras.layers.Dense(64, activation='relu')(m)
output_move = keras.layers.Dense(4, activation='sigmoid')(m)

# optimize model and compile
model = keras.Model(inputs=inp, outputs=output_move, name="pawn_star_1")
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.mse, metrics=None)
model.summary()

Model: "pawn_star_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12, 8, 8)]        0         
                                                                 
 conv1 (Conv2D)              (None, 12, 8, 64)         4672      
                                                                 
 mPool1 (MaxPooling2D)       (None, 6, 4, 64)          0         
                                                                 
 conv2 (Conv2D)              (None, 6, 4, 64)          36928     
                                                                 
 mPool2 (MaxPooling2D)       (None, 3, 2, 64)          0         
                                                                 
 flatten (Flatten)           (None, 384)               0         
                                                                 
 dense (Dense)               (None, 64)                

In [10]:
# load in training inputs and outputs
train_boards = master["oneHboards"].to_list()
train_moves = master["moves"].to_list()

# create a callback method to save best checkpoints (was not really used)
callback = keras.callbacks.ModelCheckpoint(filepath="checkpoints/cp-{epoch:04d}.hdf5",
                                           verbose=1, 
                                           save_freq=2000, 
                                           save_weights_only=True, 
                                           #save_best_only=True,
                                           monitor="val_loss", 
                                           mode="max")

# load up inputs and begin training the model
inpx = tf.convert_to_tensor(train_boards)
inpy = tf.convert_to_tensor(train_moves)
model.fit(x=inpx, 
          y=inpy, 
          batch_size=1000, 
          epochs=10, 
          callbacks=[callback],
          validation_split=0.1)

Epoch 1/10
40/40 [==============================] - 2s 45ms/step - loss: 0.0519 - val_loss: 0.0496
Epoch 2/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0480 - val_loss: 0.0468
Epoch 3/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0456 - val_loss: 0.0452
Epoch 4/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0441 - val_loss: 0.0445
Epoch 5/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0434 - val_loss: 0.0440
Epoch 6/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0427 - val_loss: 0.0443
Epoch 7/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0422 - val_loss: 0.0436
Epoch 8/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0417 - val_loss: 0.0440
Epoch 9/10
40/40 [==============================] - 2s 44ms/step - loss: 0.0413 - val_loss: 0.0440
Epoch 10/10
40/40 [==============================] - 2s 43ms/step - loss: 0.0410 - val_loss: 0.0433


In [12]:
# This code segment is about running the model on random board positions

# load stockfish to evaluate random position
stockfish = Stockfish(path="stockfish15.1/stockfish-windows-2022-x86-64-avx2.exe")

# counters for how many times the AI or Randomness had a better move, with IM referencing # of illegal moves by the AI 
AI = 0
IM = 0
Random = 0

# generate some boards from the test set
boards = random.sample(range(1, len(test_boards)), 500)


for i in boards:
    AI_board = chess.Board(test_boards[i])
    Random_board = chess.Board(test_boards[i])
    curr_eval = stockfish.get_evaluation()["value"]
    
    # try AI eval
    pos = board_to_12_array(board_to_nums(board_fen_to_8x8(AI_board.board_fen())))
    pos = np.expand_dims(pos, 0)
    move = np.array(model(pos))
    move = move * 8
    x = 1
    m = ""
    # convert model output to input for board mover from chess module
    for e in range(len(move[0])):
        temp = move[0][e]
        temp = int(round(temp,0))
        if temp == 0:
            temp = 1
        if x > 0:
            temp = chr(96+int(temp))
        x = x * (-1)
        m += str(temp)
    # if move from (e.g. e2e2 or g6g6) classify as null (illegal) move
    if m[0:2] == m[2:4]:
        m = "0000"
    m = chess.Move.from_uci(m)
    move = m
    
    # if the move was legal, play it and do the evaluations. Otherwise, classify as an illegal move and skip the comparisons
    if(move in AI_board.legal_moves):
        AI_board.push(move)
        AI_eval = stockfish.get_evaluation()["value"]
        
        rmove = random.choice([rmove for rmove in board.legal_moves])
        Random_board.push(move)
        Random_eval = stockfish.get_evaluation()["value"]
        
        
        if (curr_eval - AI_eval) <= (curr_eval - Random_eval):
            AI += 1
            #print(board)
            #print("Curr:   " + str(curr_eval))
            #print("AI:     " + str(AI_eval))
            #print("Random: " + str(Random_eval))
        else:
            Random += 1

    else:
        IM += 1
        
print("IM:     " + str(IM))
print("AI:     " + str(AI))
print("Random: " + str(Random))

IM:     453
AI:     34
Random: 13


In [13]:
# This code segment is about a random mvoe bot playing games with the model 

# make move on chess object based on given move
def make_move(move):

    #make move if move is legal, else return "not legal"
    if(move in board.legal_moves):
        board.push(move)
    else:
        return("not legal")
    
    #check if game has ended and return outcome else return move made
    if(board.is_game_over()):
        return(board.outcome())
    else:
        return(move)
    
# choose a move to play if no legal move was suggested by the model    
def choose_random_move(move):
    for legal_move in board.legal_moves:
        # check for something with 3 matches:
        legal_move = str(legal_move)
        if (move[0] == legal_move[0] and move[1] == legal_move[1] and move[2] == legal_move[2]) or (move[0] == legal_move[0] and move[2] == legal_move[2] and move[3] == legal_move[3]) or(move[0] == legal_move[0] and move[1] == legal_move[1] and move[3] == legal_move[3]) or(move[1] == legal_move[1] and move[2] == legal_move[2] and move[3] == legal_move[3]):
            return str(legal_move)
        
        # if nothing comes up, just make a random move
        else:
            return str(random.choice([move for move in board.legal_moves]))
           
    
stockfish = Stockfish(path="stockfish15.1/stockfish-windows-2022-x86-64-avx2.exe")
# counters for white winning (AI), black winning (random), or a draw
w = 0
b = 0
d = 0
AI_t = 0
Random_t = 0
num_runs = 100
for g in range(num_runs):
    #chess object
    board = chess.Board()

    #play game with random moves
    white = 1
    while not board.is_game_over():
        # if AI's turn:
        if white > 0:
            # use the model to generate a move
            pos = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
            pos = np.expand_dims(pos, 0)
            move = np.array(model(pos))
            move = move * 8
            x = 1
            # convert model output to input for board mover from chess module
            m = ""
            for e in range(len(move[0])):
                temp = move[0][e]
                temp = int(round(temp,0))
                if temp == 0:
                    temp = 1
                if x > 0:
                    temp = chr(96+int(temp))
                x = x * (-1)
                m += str(temp)
            if m[0:2] == m[2:4]:
                m = "0000"
            m = chess.Move.from_uci(m)
            move = m

            # make the move, and if it is not legal make a random move
            gameStatus = make_move(move)
            if gameStatus == "not legal":
                move = choose_random_move(str(move))
                move = chess.Move.from_uci(move)
                gameStatus = make_move(move)
        # if random's turn:
        else:
            move = random.choice([move for move in board.legal_moves])
            gameStatus = make_move(move)

        white = white * (-1)

    outcome = board.outcome()
    if outcome.winner == chess.WHITE:
        w += 1
    elif outcome.winner == chess.BLACK:
        b += 1
    else:
        d += 1
print(w)
print(b)
print(d)


11
8
81


In [ ]:
# playing chess AI ourselves:

#chess object
board = chess.Board()

#play game with random moves
white = 1
while not board.is_game_over():
    # if it is the AI's turn:
    if white > 0:
        # produce move suggestion from model
        pos = board_to_12_array(board_to_nums(board_fen_to_8x8(board.board_fen())))
        pos = np.expand_dims(pos, 0)
        move = np.array(model(pos))
        move = move * 8
        # convert model output to input for board mover from chess module
        x = 1
        m = ""
        for e in range(len(move[0])):
            temp = move[0][e]
            temp = int(round(temp,0))
            if temp == 0:
                temp = 1
            if x > 0:
                temp = chr(96+int(temp))
            x = x * (-1)
            m += str(temp)
        if m[0:2] == m[2:4]:
            m = "0000"
        m = chess.Move.from_uci(m)
        move = m
        
        # try to make the move, and if it is not legal choose a random one.
        gameStatus = make_move(move)
        if gameStatus == "not legal":
            move = choose_random_move(str(move))
            move = chess.Move.from_uci(move)
            gameStatus = make_move(move)
    # if it is out turn:
    else:
        move = input("Enter move: (e.g. e7e5)")
        move = chess.Move.from_uci(move)
        gameStatus = make_move(move)
        
    print(gameStatus)
    print(board)
        
    white = white * (-1) 

e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
Enter move: (e.g. e7e5)e7e5
e7e5
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
d1g4
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . P . Q .
. . . . . . . .
P P P P . P P P
R N B . K B N R
Enter move: (e.g. e7e5)d7d5
d7d5
r n b q k b n r
p p p . . p p p
. . . . . . . .
. . . p p . . .
. . . . P . Q .
. . . . . . . .
P P P P . P P P
R N B . K B N R
g4f4
r n b q k b n r
p p p . . p p p
. . . . . . . .
. . . p p . . .
. . . . P Q . .
. . . . . . . .
P P P P . P P P
R N B . K B N R
